### Imports

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from matplotlib.pyplot import figure

### Data Load

In [ ]:
data = pandas.read_csv('./data/Learning/brake, acc/Train_Set.csv')
test = pandas.read_csv('./data/Learning/brake, acc/Test_Set.csv')

x_train_set = data[['x','y','z']]

target = data['AccStatus']

x_test_set = test[['x','y','z']]
y_test_set = test['AccStatus']


x_train_set = np.array(x_train_set,dtype=float)          
target = np.array(target,dtype=float)

x_test_set =  np.array(x_test_set,dtype=float)
y_test_set =  np.array(y_test_set,dtype=float)

In [ ]:
def SmoothData(data):
    size = data.shape[0] 
    dim = data.shape[1]
    
    smooth_data = np.zeros((size,dim))
    
    smooth_data[0] = data[0]
    smooth_data[1] = data[1]
    smooth_data[size-1] = data[size-1]
    smooth_data[size-2] = data[size-2]
    
    for i in range(2, size-2):
        smooth_data[i] = np.average([data[i], data[i-1], data[i+1]], axis=0)
    return smooth_data

In [ ]:
x_train_set = SmoothData(x_train_set)
x_test_set = SmoothData(x_test_set)

### One-hot encoding

In [ ]:
target_onehot  = []
y_test_set_onehot = []
for i in range(len(target)):
    if(target[i]==0):
        target_onehot.append(np.array([1,0,0]))
    elif (target[i]==1):
        target_onehot.append(np.array([0,1,0]))
    elif(target[i]==2):
        target_onehot.append(np.array([0,0,1]))
        
for i in range(len(y_test_set)):
    if(y_test_set[i]==0):
        y_test_set_onehot.append(np.array([1,0,0]))
    elif (y_test_set[i]==1):
        y_test_set_onehot.append(np.array([0,1,0]))
    elif (y_test_set[i]==2):
        y_test_set_onehot.append(np.array([0,0,1]))

In [ ]:
len (x_train_set)

### LSTM one-hot

In [ ]:
model = Sequential()
model.add(LSTM(3, input_shape = (50,3), return_sequences = True))
model.add(LSTM(10, return_sequences = True))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit( 
    np.array(x_train_set[:47000]).reshape(940,50,3),
    np.array([target_onehot[i:i+50] for i in range(0,47000,50)]),
    epochs=1000,
    validation_data= (np.array(x_train_set[47000:47600]).reshape(12,50,3),
                     np.array([target_onehot[i:i+50] for i in range(47000,47600,50)])),
batch_size=64)


In [ ]:
x_test_set.shape

In [ ]:
results = model.predict(np.array(x_test_set[:14150]).reshape(283,50,3))
results = results.reshape(14150,3)

In [ ]:
result_class = []
for i in results:
    if(np.argmax(i)==0):
        result_class.append(0)
    elif(np.argmax(i)==1):
        result_class.append(1)
    elif(np.argmax(i)==2):
        result_class.append(2)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test_set[:14150], result_class, labels=[0,1], average=None)

In [ ]:
len(result_class)

In [ ]:

fig =tools.make_subplots(rows=1,cols=1)

result =  go.Scatter(
    x = list(range(0,15000)),
    y = result_class,
    mode = 'lines',
    name='model_result'
)

test =  go.Scatter(
    x = list(range(0,15000)),
    y = y_test_set,
    mode = 'lines',
    name='test'
)

fig.append_trace(result,1,1)
fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Brake,Acc Model Classification')


py.offline.plot(fig, filename='brake,acc_onehot_50_sigmoid_categorical_crossentropy')


In [ ]:

test = pandas.read_csv('./data/Learning/brake, acc/Test_Set.csv')


fig =tools.make_subplots(rows=1,cols=1)


test =  go.Scatter(
    x = list(range(0,15000)),
    y = test['x'],
    mode = 'lines',
    name='test'
)

fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Test Data')


py.offline.plot(fig, filename='brake_acc_test_data')
